In [1]:
from os import environ

# environ['HF_HOME'] = '/Volumes/DATA/hf_home'
environ["HF_HOME"] = "/mnt/workspaces/hf_home"
environ["HTTP_PROXY"] = "http://clash.internal.moqi.ai:7890"
environ["HTTPS_PROXY"] = "http://clash.internal.moqi.ai:7890"
environ["OPENAI_API_BASE"] = "http://10.1.3.28:8990/v1"
environ["OPENAI_API_KEY"] = "sk-qB3QC4y6OtIDASTcE10c06A9F24c462eA55f95867920Df8e"


from retrieval_qa_benchmark.models import *
from retrieval_qa_benchmark.datasets import *
from retrieval_qa_benchmark.transforms import *
from retrieval_qa_benchmark.protocols import *
from retrieval_qa_benchmark.utils.registry import REGISTRY
from retrieval_qa_benchmark.utils.profiler import PROFILER

print(str(REGISTRY))

/home/fangruil/anaconda3/envs/yongqij/lib/python3.9/site-packages/pydantic/_internal/_fields.py:127: UserWarning: Field "model_name" has conflict with protected namespace "model_".

You may be able to resolve this warning by setting `model_config['protected_namespaces'] = ()`.
  warnings.warn(


Datasets:
	hotpot_qa
	mmlu
	wikihop
Transforms:
	base
	add_myscale_retrieval
	mcma_prompt
	mcsa_prompt
	mcsa_fewshot_mmlu
Models:
	remote-llm
	gpt35
	chatgpt35
Evaluators:
	mcma
	mcsa


/home/fangruil/anaconda3/envs/yongqij/lib/python3.9/site-packages/pydantic/_internal/_fields.py:127: UserWarning: Field "model_name" has conflict with protected namespace "model_".

You may be able to resolve this warning by setting `model_config['protected_namespaces'] = ()`.
  warnings.warn(


In [2]:
import yaml
from retrieval_qa_benchmark.utils.config import load
config = load(open("../config/test.yaml"))
evaluator = REGISTRY.Evaluators["mcsa"].from_config(config)
# print(evaluator.transform)

Converting dataset...: 100%|██████████| 324/324 [00:00<00:00, 6925.67it/s]
[nltk_data] Downloading package stopwords to
[nltk_data]     /home/yongqij/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /home/yongqij/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/yongqij/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to /home/yongqij/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [4]:
data = evaluator.transform(evaluator.dataset[0])
# print(evaluator.transform(evaluator.dataset[0]).raw_question)
# evaluator.dataset.eval_set = evaluator.dataset.eval_set[:5]
data

QARecord(id='4e8db4a928333356588d2b89477e5d5bcb95532cd8afa355456ac94e13fd1849', question='Unlike most other early civilizations, Minoan culture shows little evidence of:', answer='conspicuous consumption by elites.', type='MCSA', context=None, choices=['trade.', 'warfare.', 'the development of a common religion.', 'conspicuous consumption by elites.'], stack=[])

In [7]:
import yaml

config = yaml.safe_load(open("../config/mmlu-llama2-remote.yaml"))
evaluator = REGISTRY.Evaluators["mcsa"].from_config(config)
print(evaluator.transform)

load index...
load mpnet model...
load Colbert model...
chain=[MultipleChoiceTransform(sep_chr='\n', prompt_prefix='The following are multiple choice questions (with answers)\n', prompt_suffix='Answer: '), AddContextTransform(sep_chr='\n', prompt_prefix='Context:', context=['1', '2'], replace=False), AddMyScaleRetrievalTransform(sep_chr='\n', prompt_prefix='Context:', replace=False, num_filtered=100, num_selected=5, with_title=True, rank_dict={'mpnet': 30, 'bm25': 40}, index_path='/mnt/workspaces/yongqij/evals/data/indexes/Cohere_mpnet/IVFSQ_L2.index', model_name='paraphrase-multilingual-mpnet-base-v2', dataset_name=['Cohere/wikipedia-22-12-en-embeddings'])]


In [5]:
print(str(PROFILER))

{'id': '4e8db4a928333356588d2b89477e5d5bcb95532cd8afa355456ac94e13fd1849', 'answer': 'conspicuous consumption by elites.', 'type': 'MCSA', 'raw_question': 'Unlike most other early civilizations, Minoan culture shows little evidence of:', 'context': ['1', '2'], 'choices': ['trade.', 'warfare.', 'the development of a common religion.', 'conspicuous consumption by elites.'], 'question': 'The following are multiple choice questions (with answers)\n\nContext:\n[1] 1\n[2] 2\n\nUnlike most other early civilizations, Minoan culture shows little evidence of:\nA. trade.\nB. warfare.\nC. the development of a common religion.\nD. conspicuous consumption by elites.\nAnswer: '}
['Minoan civilization | According to Arthur Evans, a "Minoan peace" ("Pax Minoica") existed; there was little internal armed conflict in Minoan Crete until the Mycenaean period. However, it is difficult to draw hard-and-fast conclusions from the evidence and Evans\' idealistic view has been questioned.', 'Minoan civilization 

In [ ]:
matched = evaluator()

In [ ]:
print(str(PROFILER))

In [ ]:
with open("results.with-retrieval.jsonl", "w") as f:
    f.write("\n".join([r.model_dump_json() for r in matched[1]]))

In [ ]:
for dname, dset in REGISTRY.Datasets.items():
    print(f"Loading {dname}...")
    dset.build()

In [ ]:
from retrieval_qa_benchmark.schema import QAPrediction

mismatched = [pred for pred in matched[1] if pred.matched]

In [ ]:
with open("results.test.jsonl", "w") as f:
    f.write("\n".join([r.model_dump_json() for r in mismatched]))